In [ ]:
import openai
import os
import pandas

openai.api_key = str(os.getenv('GPT_TOKEN'))

dataset = pandas.DataFrame(columns = ['sentence', 'sentiment'])
iterations = 400

for i in range(iterations):
    positive_response = openai.Completion.create(
        model='text-davinci-002',
        prompt='Write an example of a sentence with a positive sentiment. The sentence must be grammatically correct.\n\n',
        temperature=1,
        max_tokens=256,
        frequency_penalty=0,
        presence_penalty=0
    )
    negative_response = openai.Completion.create(
        model='text-davinci-002',
        prompt='Write an example of a sentence with a negative sentiment. The sentence must be grammatically correct.\n\n',
        temperature=1,
        max_tokens=256,
        frequency_penalty=0,
        presence_penalty=0
    )
    positive_response = {
        'sentence' : positive_response['choices'][0]['text'].replace('\n',''),
        'sentiment' : 'positive'
    }
    negative_response = {
        'sentence' : negative_response['choices'][0]['text'].replace('\n',''),
        'sentiment' : 'negative'
    }
    dataset = dataset.append(positive_response, ignore_index = True)
    dataset = dataset.append(negative_response, ignore_index = True)
    

dataset.to_csv(os.path.join(os.path.curdir, 'sentiment_sentence_dataset.csv'))
       
               

In [ ]:
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

dataset = pandas.read_csv('sentiment_sentence_dataset.csv', index_col = 0)

dataset['sentiment'] = LabelEncoder().fit_transform(dataset['sentiment'])

X = TfidfVectorizer().fit_transform(dataset['sentence'])
y = dataset['sentiment']

train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 42)

classifier = SVC()
classifier.fit(train_x, train_y)

predictions = classifier.predict(test_x)

print(confusion_matrix(test_y, predictions))